<a href="https://colab.research.google.com/github/RomainCendre/DSTest/blob/main/deepsense_generative_takehome_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Data modeling

Given the following [data-set](https://gist.githubusercontent.com/alephd/e3899eaae0e7869e9c230f6a0fe944e5/raw/deab4f93a9c15224ac118d46533d9ea9700d206a/2019_data_science_test.json).

### 1. Can you generate 10k rows of new data from the same distribution?

You can use the code below to load the data.

Please explain your approach. (Hints: you may exploit some techniques using neural networks)

Please provide the code that generates the data in this notebook.



In [ ]:
### Answer


### 2. How could prior-knowledge help you in this task?

In [1]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as img
import torch
import sklearn as skl
import numpy as np

# Read the Gist
res = requests.get("https://gist.githubusercontent.com/IdrissMg/695f423af0f0aaffd8d69bc403932c6b/raw/6f3192bbcc328f2aa53ae2faef0cefb825564ea9/data_science_test.json")
data = np.array(res.json())

In [2]:
# The data-set is properly loaded
print(data[:10,:])

[[ 0.35550847 -0.16861307 -0.03649862  0.19952643 -0.11887184  0.29424839
  -0.28051336]
 [ 0.4260113  -0.21258807 -0.01852221  0.22457905 -0.15877081  0.32463416
  -0.37767599]
 [ 0.29133138 -0.1376825   0.0477822   0.36069595 -0.26262892  0.36821576
  -0.22965423]
 [ 0.31290507 -0.15962714  0.05445522  0.33733013 -0.2346064   0.2984015
  -0.20969495]
 [ 0.30500781 -0.13593187 -0.01060943  0.28193989 -0.23838613  0.33592651
  -0.22018852]
 [ 0.31491301 -0.14003621  0.00660586  0.275029   -0.18690363  0.33443304
  -0.27883085]
 [ 0.39339591 -0.13173083  0.0158673   0.18128972 -0.15204574  0.3052976
  -0.34210036]
 [ 0.3136345  -0.15933352  0.00958527  0.28161228 -0.1882738   0.30471432
  -0.2638125 ]
 [ 0.33660039 -0.17667822  0.0593603   0.31968113 -0.28692315  0.38236689
  -0.26209681]
 [ 0.3190583  -0.13005376  0.02983905  0.16937195 -0.13130105  0.23831343
  -0.27769289]]
